In [22]:
import sys
import os
import pandas as pd
import numpy as np
from exploringShipLogbooks.basic_utils import extract_logbook_data

       
def is_new_trip(position,trip_id):
    index = 0
    trip = [0]*len(position['Date'])
    trip [0] = trip_id
    while index < len(position['Date'])-1:
        index += 1
        if index ==len(position['Date'])-1:
            if (position.iat[index,8] - position.iat[index-1,8] == 8870) \
                or (position.iat[index,8] - position.iat[index-1,8]<100 \
                    and position.iat[index,3] == position.iat[index-1,3] \
                    and position.iat[index,4] == position.iat[index-1,4]):
                num_data = trip.count(trip_id)
                if trip.count(trip_id) < 5:
                    for i in range(index-1, index-num_data-1, -1):
                        trip[i]=0
                    continue
                else:
                    trip[index] = trip_id
                    trip_id += 1
                    continue        
        if (position.iat[index,8] - position.iat[index-1,8] == 8870) \
            or (position.iat[index,8] - position.iat[index-1,8]<100 \
            and position.iat[index,3] == position.iat[index-1,3] \
            and position.iat[index,4] == position.iat[index-1,4]):
            trip[index] = trip_id
        else:
            ##delete trips that is too short
            num_data = trip.count(trip_id)
            if trip.count(trip_id) < 5:
                for i in range(index-1, index-num_data-1, -1):
                    trip[i]=0  ## the data point would be marked 0
            else:
                trip_id += 1
                trip[index] = trip_id
    position.insert(9, "Trip", trip)
    return trip_id


def split_trip(raw_data):
    #shipname = pd.value_counts(raw_data["ShipName"]).head(10).keys()
    shipname = list(pd.value_counts(raw_data["ShipName"]).keys()) #949 shipnames
    
    shipnames_to_drop = ["SAN CARLOS", "LA SUERTE", "LABORDE", "CONOCIDO-26", 
                         "NaSa DEL ROSARIO,STMA TRINIDAD", "Vrouwe Margaretha Maria", 
                         "EL MARLLORQUÍN", "LA ORBE", "LA JUNO", "DESCONOCIDO-01",
                         "DESCONOCIDO-08", "DESCONOCIDO-08", "DESCONOCIDO-21", "DESCONOCIDO-11"]
    #Some ships could cause error and need to be removed manually
    for name in shipnames_to_drop:
        shipname = [x for x in shipname if name not in x]
  
    position_all = []
    trip_id = 1

    for ship in shipname:
        position = raw_data[raw_data["ShipName"]==ship][pd.notnull(raw_data["Lat3"])][pd.notnull(raw_data["Lon3"])] \
                            [["Lat3", "Lon3", "VoyageFrom", "VoyageTo", 
                              "Year", "Month", "Day", "UTC", "Nationality", "RecID"]]
        position = position.sort_values(by='UTC', ascending=True)
        position.insert(8, "Date", position['UTC'])
        position['Date'] = position['Date'] // 100

        trip_id = is_new_trip(position,trip_id)
        position_each_ship = list(zip([ship]*len(position["Lat3"]), position["Nationality"], 
                                      position["VoyageFrom"], position["VoyageTo"],
                                      position["Year"], position["Date"], position["Lat3"],
                                      position["Lon3"], position["RecID"], position["Trip"]))

        position_all += position_each_ship

    return position_all



    



            

In [16]:
raw_data = extract_logbook_data('CLIWOC15.csv')

C:\Users\Emma\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3006: DtypeWarning: Columns (5,6,7,8,11,13,18,19,23,24,25,26,28,29,30,34,35,38,43,44,46,73,77,81,82,84,85,87,88,94,96,97,98,99,111,114,116,119,120,122,124,125,127,129,131,133,135,137,140) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [24]:
split_trip(raw_data)

C:\Users\Emma\Anaconda3\lib\site-packages\pandas\core\frame.py:1997: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  "DataFrame index.", UserWarning)


[('PRINCIPE',
  'Spanish',
  'LA CORUÑA',
  'LA HABANA',
  1765,
  17650503,
  43.783299999999997,
  -10.35,
  54565,
  1),
 ('PRINCIPE',
  'Spanish',
  'LA CORUÑA',
  'LA HABANA',
  1765,
  17650504,
  43.066699999999997,
  -12.316700000000001,
  54566,
  1),
 ('PRINCIPE',
  'Spanish',
  'LA CORUÑA',
  'LA HABANA',
  1765,
  17650505,
  42.033299999999997,
  -14.083299999999999,
  54567,
  1),
 ('PRINCIPE',
  'Spanish',
  'LA CORUÑA',
  'LA HABANA',
  1765,
  17650506,
  41.833300000000001,
  -15.666700000000001,
  54270,
  1),
 ('PRINCIPE',
  'Spanish',
  'LA CORUÑA',
  'LA HABANA',
  1765,
  17650507,
  40.350000000000001,
  -16.833300000000001,
  54271,
  1),
 ('PRINCIPE',
  'Spanish',
  'LA CORUÑA',
  'LA HABANA',
  1765,
  17650508,
  39.450000000000003,
  -18.283300000000001,
  54272,
  1),
 ('PRINCIPE',
  'Spanish',
  'LA CORUÑA',
  'LA HABANA',
  1765,
  17650509,
  38.333300000000001,
  -20.649999999999999,
  54470,
  1),
 ('PRINCIPE',
  'Spanish',
  'LA CORUÑA',
  'LA HABANA

In [23]:
df = pd.DataFrame(split_trip(raw_data), columns=["ShipName", "Nationality", "VoyageFrom",
                                                 "VoyageTo", "Year","Date", "Latitude", 
                                                 "Longtitude", "RecID","Trip"])


C:\Users\Emma\Anaconda3\lib\site-packages\pandas\core\frame.py:1997: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  "DataFrame index.", UserWarning)


In [14]:
df.head()

,ShipName,Nationality,VoyageFrom,VoyageTo,Year,Date,Latitude,Longtitude,RecID,Trip
0,PRINCIPE,Spanish,LA CORUÑA,LA HABANA,1765,17650503,43.7833,-10.3500,54565,1
1,PRINCIPE,Spanish,LA CORUÑA,LA HABANA,1765,17650504,43.0667,-12.3167,54566,1
2,PRINCIPE,Spanish,LA CORUÑA,LA HABANA,1765,17650505,42.0333,-14.0833,54567,1
3,PRINCIPE,Spanish,LA CORUÑA,LA HABANA,1765,17650506,41.8333,-15.6667,54270,1
4,PRINCIPE,Spanish,LA CORUÑA,LA HABANA,1765,17650507,40.3500,-16.8333,54271,1


In [ ]:
#raw_data.insert(141, "Trip_ID",0)
#trip_id = [0]*len(raw_data["Trip_ID"])
trip_id = [0]*len(raw_data["RecID"]) #Set a new column for trip_id
for i in range(len(raw_data["RecID"])):
    try: #if raw_data's RecID macth RecID in df, then return the trip_id for that line
        trip_id[i]=df.loc[df[df["RecID"]==raw_data.iat[i,0]]["Trip"].keys()[0],"Trip"] 
    except:
        pass

#print(trip_id[-50:])



In [35]:
print(df[df["RecID"]==548]["Trip"])   #Test for datatype of this sentence, not necessary for main function

199168    3470
Name: Trip, dtype: int64


In [ ]:
raw_data.insert(141, "Trip_ID", trip) ##add the trip id colunmn to the end of raw data